# Fall 2025 Data Science Track: Week 2 - Data Cleaning Exercise

## Packages, Packages, Packages!

Import *all* the things here! You need the standard stuff: `pandas` and `numpy`.

If you got more stuff you want to use, add them here too. 🙂

In [59]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
# numpy not used 

# used to pretty print dictionaries with many keys
from pprint import pprint

## Introduction

With the packages out of the way, now you will be working with the following data sets:

* `food_coded.csv`: [Food choices](https://www.kaggle.com/datasets/borapajo/food-choices?select=food_coded.csv) from Kaggle
* `Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv`: [Ask A Manager Salary Survey 2021 (Responses)](https://docs.google.com/spreadsheets/d/1IPS5dBSGtwYVbjsfbaMCYIWnOuRmJcbequohNxCyGVw/view?&gid=1625408792) as *Tab Separated Values (.tsv)* from Google Docs

Each one poses different challenges. But you’ll―of course―overcome them with what you learned in class! 😉

## Food Choices Data Set

### Load the Data

In [60]:
food_data_set_path = '../data/food_coded.csv'
food_df = pd.read_csv(food_data_set_path)

### Explore the Data

How much data did you just load?

In [61]:
row_count, col_count = food_df.shape
print(f"Rows: {row_count}\nColumns: {col_count}")
food_df.info()

# ** Quantity of data loaded ** #
# 125 rows
# 61 cols
# 59.7+ KB memory

Rows: 125
Columns: 61
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 61 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   GPA                           123 non-null    object 
 1   Gender                        125 non-null    int64  
 2   breakfast                     125 non-null    int64  
 3   calories_chicken              125 non-null    int64  
 4   calories_day                  106 non-null    float64
 5   calories_scone                124 non-null    float64
 6   coffee                        125 non-null    int64  
 7   comfort_food                  124 non-null    object 
 8   comfort_food_reasons          123 non-null    object 
 9   comfort_food_reasons_coded    106 non-null    float64
 10  cook                          122 non-null    float64
 11  comfort_food_reasons_coded.1  125 non-null    int64  
 12  cuisine                       108 non-null

What are the columns and their types in this data set?

In [62]:
# Show the column names and their types.
pprint(food_df.dtypes.to_dict())

{'GPA': dtype('O'),
 'Gender': dtype('int64'),
 'breakfast': dtype('int64'),
 'calories_chicken': dtype('int64'),
 'calories_day': dtype('float64'),
 'calories_scone': dtype('float64'),
 'coffee': dtype('int64'),
 'comfort_food': dtype('O'),
 'comfort_food_reasons': dtype('O'),
 'comfort_food_reasons_coded': dtype('float64'),
 'comfort_food_reasons_coded.1': dtype('int64'),
 'cook': dtype('float64'),
 'cuisine': dtype('float64'),
 'diet_current': dtype('O'),
 'diet_current_coded': dtype('int64'),
 'drink': dtype('float64'),
 'eating_changes': dtype('O'),
 'eating_changes_coded': dtype('int64'),
 'eating_changes_coded1': dtype('int64'),
 'eating_out': dtype('int64'),
 'employment': dtype('float64'),
 'ethnic_food': dtype('int64'),
 'exercise': dtype('float64'),
 'father_education': dtype('float64'),
 'father_profession': dtype('O'),
 'fav_cuisine': dtype('O'),
 'fav_cuisine_coded': dtype('int64'),
 'fav_food': dtype('float64'),
 'food_childhood': dtype('O'),
 'fries': dtype('int64'),
 '

In [63]:
food_df.dtypes.value_counts()

int64      27
float64    20
object     14
Name: count, dtype: int64

### Clean the Data

Perhaps we’d like to know more another day, but the team is really interested in just the relationship between calories (`calories_day`) and weight. …and maybe gender.

Can you remove the other columns?

In [64]:
# Get subset of the data we are interested in
print(food_df.columns)
food_min_df = food_df[["calories_day", "weight", "Gender"]]
food_min_df.head()

Index(['GPA', 'Gender', 'breakfast', 'calories_chicken', 'calories_day',
       'calories_scone', 'coffee', 'comfort_food', 'comfort_food_reasons',
       'comfort_food_reasons_coded', 'cook', 'comfort_food_reasons_coded.1',
       'cuisine', 'diet_current', 'diet_current_coded', 'drink',
       'eating_changes', 'eating_changes_coded', 'eating_changes_coded1',
       'eating_out', 'employment', 'ethnic_food', 'exercise',
       'father_education', 'father_profession', 'fav_cuisine',
       'fav_cuisine_coded', 'fav_food', 'food_childhood', 'fries', 'fruit_day',
       'grade_level', 'greek_food', 'healthy_feeling', 'healthy_meal',
       'ideal_diet', 'ideal_diet_coded', 'income', 'indian_food',
       'italian_food', 'life_rewarding', 'marital_status',
       'meals_dinner_friend', 'mother_education', 'mother_profession',
       'nutritional_check', 'on_off_campus', 'parents_cook', 'pay_meal_out',
       'persian_food', 'self_perception_weight', 'soup', 'sports', 'thai_food',
       

,calories_day,weight,Gender
0,NaN,187,2
1,3.0,155,1
2,4.0,I'm not answering this.,1
3,3.0,"Not sure, 240",1
4,2.0,190,1


What about `NaN`s? How many are there?

In [65]:
# determine the total and per column count of missing values
missing = food_min_df.isna().sum()

print(f"Total missing values: {missing.sum()}")
print(missing)

Total missing values: 21
calories_day    19
weight           2
Gender           0
dtype: int64


We gotta remove those `NaN`s―the entire row.

In [66]:
# remove rows that have missing values
food_min_df = food_min_df.dropna(ignore_index=True)
print(f"Dropped {row_count - len(food_min_df)} rows")

# must mean that 21 different rows each had one missing value

Dropped 21 rows


But what about the weird non-numeric values in the column obviously meant for numeric data?

Notice the data type of that column from when you got the types of all the columns?

If only we could convert the column to a numeric type and drop the rows with invalid values. 🤔

In [67]:
row_count = len(food_min_df)

food_min_df["weight"] = pd.to_numeric(food_min_df["weight"], errors="coerce")
food_min_df = food_min_df.dropna(ignore_index=True)

print(f"Dropped {row_count - len(food_min_df)} rows.")
food_min_df.dtypes

Dropped 3 rows.


calories_day    float64
weight          float64
Gender            int64
dtype: object

Now this data seems reasonably clean for our purposes! 😁

Let’s save it somewhere to be shipped off to another teammate. 💾

In [68]:
# rename columns first
food_min_df = food_min_df.rename(str.lower, axis="columns")

# export to csv
food_min_df.to_csv("../data/food_coded_cleaned.csv", index=False)

## Ask a Manager Salary Survey 2021 (Responses) Data Set

### Load the Data

In [69]:
# Load the Ask A Manager Salary Survey 2021 (Responses) data set into a variable (e.g., df_salary).

# salary_df = pd.read_csv("../data/Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv", sep="\t")
salary_df = pd.read_csv("../data/manager_salary_survey_2021.tsv", sep="\t")

Was that hard? 🙃

#### rename the file to something that is better for all systems.  
* No spaces in filename (can use '_')
* all lower case

### Explore

You know the drill.

How much data did you just load?

In [70]:
n_rows, n_cols = salary_df.shape
print(f"Rows: {n_rows}\nColumns: {n_cols}")
salary_df.info()

# ** Quantity of data loaded ** #
# Rows: 28062
# Columns: 18
# Memory: 3.9+ MB

Rows: 28062
Columns: 18
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28062 entries, 0 to 28061
Data columns (total 18 columns):
 #   Column                                                                                                                                                                                                                                Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                                --------------  -----  
 0   Timestamp                                                                                                                                                                                                                             28062 non-null  object 
 1   How old are you?                                                                     

What are the columns and their types?

In [71]:
salary_df.dtypes

Timestamp                                                                                                                                                                                                                                object
How old are you?                                                                                                                                                                                                                         object
What industry do you work in?                                                                                                                                                                                                            object
Job title                                                                                                                                                                                                                                object
If your job title needs additional conte

Oh… Ugh! Give these columns easier names to work with first. 🙄

In [72]:
salary_df.rename({
       "Timestamp": "timestamp", 
       "How old are you?": "age", 
       "What industry do you work in?": "industry",
       "Job title": "job_title",
       "If your job title needs additional context, please clarify here:": "job_title_context",
       "What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)": "annual_salary",
       "How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.": "additional_monetary_compensation",
       "Please indicate the currency": "currency",
       'If "Other," please indicate the currency here: ': "other_currency",
       "If your income needs additional context, please provide it here:": "income_context",
       "What country do you work in?": "country",
       "If you're in the U.S., what state do you work in?": "state",
       "What city do you work in?": "city",
       "How many years of professional work experience do you have overall?": "total_yoe",
       "How many years of professional work experience do you have in your field?": "field_yeo",
       "What is your highest level of education completed?": "highest_ed_level",
       "What is your gender?": "gender", 
       "What is your race? (Choose all that apply.)": "race"
}, axis="columns", inplace=True)
salary_df.dtypes

timestamp                            object
age                                  object
industry                             object
job_title                            object
job_title_context                    object
annual_salary                        object
additional_monetary_compensation    float64
currency                             object
other_currency                       object
income_context                       object
country                              object
state                                object
city                                 object
total_yoe                            object
field_yeo                            object
highest_ed_level                     object
gender                               object
race                                 object
dtype: object

It’s a lot, and that should not have been easy. 😏

You’re going to have a gander at the computing/tech subset first because thats *your* industry. But first, what value corresponds to that `industry`?

In [73]:
# List the unique industries and a count of their instances.
salary_df["industry"].value_counts()

industry
Computing or Tech                        4699
Education (Higher Education)             2464
Nonprofits                               2419
Health care                              1896
Government and Public Administration     1889
                                         ... 
Warehousing                                 1
Education (Early Childhood Education)       1
SAAS                                        1
Health and Safety                           1
Aerospace Manufacturing                     1
Name: count, Length: 1219, dtype: int64

That value among the top 5 is what you’re looking for innit? Filter out all the rows not in that industry and save it into a new dataframe. 

In [74]:
tech_filter = salary_df["industry"] == "Computing or Tech"
salary_tech_df = salary_df[tech_filter]
print(salary_tech_df.shape)

(4699, 18)


Do a sanity check to make sure that the only values you kept are the one you are filtered for.  

In [75]:
# Sanity Check 
salary_tech_df["industry"].value_counts()

industry
Computing or Tech    4699
Name: count, dtype: int64

We are very interested in salary figures. But how many dollars 💵 is a euro 💶 or a pound 💷? That sounds like a problem for another day. 🫠

For now, let’s just look at U.S. dollars (`'USD'`).

In [76]:
salary_tech_df["currency"].value_counts()

currency
USD        3777
GBP         332
CAD         231
EUR         213
AUD/NZD      70
Other        47
SEK          13
CHF          10
JPY           4
ZAR           2
Name: count, dtype: int64

In [77]:
# Filtery filter for just the jobs that pay in USD!
usd_filter = salary_tech_df["currency"] == "USD"
salary_tech_us_df = salary_tech_df[usd_filter]

print(salary_tech_us_df.shape)

(3777, 18)


What we really want know is how each U.S. city pays in tech. What value in `country` represents the United States of America?

In [78]:
# We did filter for USD, so if we do a count of each unique country in descending count order, the relevant value(s) should show up at the top.

print(salary_tech_us_df["country"].unique())
# A LOT...

salary_tech_us_df["country"].value_counts()

['US' 'United States' 'USA' 'United States of America' 'United States '
 'U.S. ' 'US ' 'usa' 'United states' 'United State of America' 'USA '
 'U.S.' 'us' 'Usa' 'united states' 'Uniyed states' 'U.S.A.' 'America'
 'Puerto Rico ' 'U.s.' ' US' 'Unite States' 'Us' 'United State'
 'Australia' 'Cuba' 'Danmark' 'Canada' 'Italy' 'International '
 'UnitedStates' 'Israel' 'India' 'United Stated' 'U.S' 'Usa '
 'Remote (philippines)' 'Singapore' 'United States Of America' 'Spain'
 'United States of America ' 'Uruguay' 'Brazil' 'Mexico' 'United Kingdom'
 'Canada ' 'United Stateds' 'ISA' 'singapore' 'Pakistan' 'united States'
 'New Zealand' 'Poland' 'France' 'Netherlands' 'China' 'San Francisco'
 'Romania' 'United STates' 'Japan' 'U.S.A' 'United Stares'
 'United States of america' 'Australian ' 'Denmark' 'Jamaica'
 'Unites States' 'Thailand' 'U.S.A ' 'Colombia' 'denmark' 'Ghana'
 'Nigeria ' 'ss' 'Nigeria' 'Burma']


country
United States               1576
USA                         1222
US                           412
U.S.                         108
United States of America      90
                            ... 
Ghana                          1
Nigeria                        1
ss                             1
Nigeria                        1
Burma                          1
Name: count, Length: 76, dtype: int64

### Clean the Data

Well, we can’t get our answers with what we currently have, so you’ll have to make some changes.

Let’s not worry about anything below the first 5 values for now. Convert the top 5 to a single canonical value―say, `'US'`, which is nice and short.

In [79]:
# Replace them all with 'US'.
us_labels = ["United States", "USA", "US", "U.S.", "United States of America"]
salary_tech_us_df.loc[:, "country"] = salary_tech_us_df["country"].apply(lambda x: "US" if x in us_labels else x)

Have a look at the count of each unique country again now.

In [80]:
# Count again.
salary_tech_us_df["country"].value_counts()

country
US                3408
United States       68
Usa                 59
USA                 56
usa                 28
                  ... 
Ghana                1
Nigeria              1
ss                   1
Nigeria              1
Burma                1
Name: count, Length: 72, dtype: int64

Did you notice anything interesting?

In [81]:
# BONUS CREDIT: resolve [most of] those anomalous cases too without exhaustively taking every variant literally into account.

# this approach cannot handle typos
base_us_labels = ["us", "usa", "unitedstates", "unitedstatesofamerica"]
clean = lambda x: x.replace(" ", "").replace(".", "").lower() if type(x) == str else x
salary_tech_us_df.loc[:, "country"] = salary_tech_us_df["country"].apply(lambda x: "US" if clean(x) in base_us_labels else x)

In [82]:
# BONUS CREDIT: if you’ve resolved it, let’s see how well you did by counting the number of instances of each unique value.
salary_tech_us_df["country"].value_counts()

country
US                         3712
Israel                        5
Canada                        4
United State of America       2
Unite States                  2
Australia                     2
India                         2
Brazil                        2
Spain                         2
New Zealand                   2
United Kingdom                2
France                        2
Poland                        2
Cuba                          1
United State                  1
Puerto Rico                   1
Uniyed states                 1
Remote (philippines)          1
United Stated                 1
Italy                         1
International                 1
Danmark                       1
America                       1
Uruguay                       1
Singapore                     1
United Stateds                1
Canada                        1
Mexico                        1
ISA                           1
Pakistan                      1
singapore                     1


It’s looking good so far. Let’s find out the minimum, mean, and maximum (in that order) salary by state, sorted by the mean in descending order.

In [83]:
# Find the minimum, mean, and maximum salary in USD by U.S. state.
salary_tech_us_df.groupby("state")["annual_salary"].agg(["min", "mean", "max"]).sort_values(by="mean", ascending=False)

TypeError: agg function failed [how->mean,dtype->object]

Well, pooh! We forgot that `salary` isn’t numeric. Something wrong must be fixed.

In [ ]:
# convert annual_salary to numeric data 
salary_tech_us_df["annual_salary"] = pd.to_numeric(salary_tech_us_df["annual_salary"].apply(lambda x: x.replace(",", "") if type(x) == str else x))

Let’s try that again.

In [ ]:
salary_tech_us_df.groupby("state")["annual_salary"].agg(["min", "mean", "max"]).sort_values(by="mean", ascending=False)

,min,mean,max
state,,,
"Michigan, Texas, Washington",340000,340000.000000,340000
"California, Oregon",200000,200000.000000,200000
"California, Colorado",176000,176000.000000,176000
"Georgia, Massachusetts",175000,175000.000000,175000
Florida,28800,157457.232143,2600000
...,...,...,...
Louisiana,35360,83269.090909,150000
"Massachusetts, Pennsylvania",83000,83000.000000,83000
"California, Maryland",81500,81500.000000,81500


That did the trick! Now let’s narrow this to data 2021 and 2022 just because (lol). *(Hint: that timestamp column may not be a temporal type right now.)*

In [ ]:
# Filter the data to within 2021, 2022, or 2023, saving the DataFrame to a new variable, and generate the summary again.
salary_tech_us_df["timestamp"] = pd.to_datetime(salary_tech_us_df["timestamp"])

years = [2021, 2022]
year_filter = salary_tech_us_df["timestamp"].dt.year.isin(years)
salary_tech_us_2021_2022_df = salary_tech_us_df[year_filter]

print(f"Filtered out {len(salary_tech_us_df) - len(salary_tech_us_2021_2022_df)} rows.")
salary_tech_us_2021_2022_df.groupby("state")["annual_salary"].agg(["min", "mean", "max"]).sort_values(by="mean", ascending=False)

Filtered out 27 rows.


,min,mean,max
state,,,
"Michigan, Texas, Washington",340000,340000.000000,340000
"California, Oregon",200000,200000.000000,200000
"California, Colorado",176000,176000.000000,176000
"Georgia, Massachusetts",175000,175000.000000,175000
"Alabama, District of Columbia",156000,156000.000000,156000
...,...,...,...
Louisiana,35360,83269.090909,150000
"Massachusetts, Pennsylvania",83000,83000.000000,83000
Arkansas,55000,81682.300000,144000


## Bonus

Clearly, we do not have enough data to produce useful figures for the level of specificity you’ve now reached. What do you notice about Delaware and West Virginia?

Let’s back out a bit and return to `df_salary` (which was the loaded data with renamed columns but *sans* filtering).

In [ ]:
# copying this here to have everything in one place
salary_df = pd.read_csv("../data/manager_salary_survey_2021.tsv", sep="\t")
salary_df.rename({
       "Timestamp": "timestamp", 
       "How old are you?": "age", 
       "What industry do you work in?": "industry",
       "Job title": "job_title",
       "If your job title needs additional context, please clarify here:": "job_title_context",
       "What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)": "annual_salary",
       "How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.": "additional_monetary_compensation",
       "Please indicate the currency": "currency",
       'If "Other," please indicate the currency here: ': "other_currency",
       "If your income needs additional context, please provide it here:": "income_context",
       "What country do you work in?": "country",
       "If you're in the U.S., what state do you work in?": "state",
       "What city do you work in?": "city",
       "How many years of professional work experience do you have overall?": "total_yoe",
       "How many years of professional work experience do you have in your field?": "field_yeo",
       "What is your highest level of education completed?": "highest_ed_level",
       "What is your gender?": "gender", 
       "What is your race? (Choose all that apply.)": "race"
}, axis="columns", inplace=True)

### Bonus #0

Apply the same steps as before to `df_salary`, but do not filter for any specific industry. Do perform the other data cleaning stuff, and get to a point where you can generate the minimum, mean, and maximum by state.

In [ ]:
# clean up country column references to the US
us_labels = ["us", "usa", "unitedstates", "unitedstatesofamerica"]
us_label = "US"
clean = lambda x: x.replace(" ", "").replace(".", "").lower() if type(x) == str else x
salary_df["country"] = salary_df["country"].apply(lambda x: us_label if clean(x) in us_labels else x)

# select US data only
us_filter = salary_df["country"] == us_label
salary_us_df = salary_df[us_filter]

# convert annual_salary to numeric data
salary_us_df["annual_salary"] = pd.to_numeric(salary_us_df["annual_salary"].str.replace(",", ""))

# convert timestamp to datetime
salary_us_df["timestamp"] = pd.to_datetime(salary_us_df["timestamp"])

# get data for 2021 and 2022
years = [2021, 2022]
year_filter = salary_us_df["timestamp"].dt.year.isin(years)
salary_us_2021_2022_df = salary_us_df[year_filter]

# generate state-by-state summary stats in descending order of mean salary
summary_df = salary_us_2021_2022_df.groupby("state")["annual_salary"].agg(["min", "mean", "max"]).sort_values(by="mean", ascending=False)
summary_df.head()

,min,mean,max
state,,,
"Michigan, Texas, Washington",340000,340000.0,340000
"Indiana, Ohio",245000,245000.0,245000
"Colorado, Nevada",190000,190000.0,190000
"California, Texas",185000,185000.0,185000
"California, Montana",185000,185000.0,185000


### Bonus #1

This time, format the table output nicely (*$12,345.00*) without modifying the values in the `DataFrame`. That is, `df_salary` should be identical before versus after running your code.

(*Hint: if you run into an error about `jinja2` perhaps you need to `pip install` something.*)

In [ ]:
money_format = "${:,.2f}"
df_formatting = {
    "min": money_format,
    "mean": money_format, 
    "max": money_format
}

summary_df.style.format(df_formatting)

,min,mean,max
state,,,
"Michigan, Texas, Washington","$340,000.00","$340,000.00","$340,000.00"
"Indiana, Ohio","$245,000.00","$245,000.00","$245,000.00"
"Colorado, Nevada","$190,000.00","$190,000.00","$190,000.00"
"California, Texas","$185,000.00","$185,000.00","$185,000.00"
"California, Montana","$185,000.00","$185,000.00","$185,000.00"
"Georgia, Massachusetts","$175,000.00","$175,000.00","$175,000.00"
"Alabama, District of Columbia","$156,000.00","$156,000.00","$156,000.00"
"Arizona, California","$90,000.00","$152,500.00","$215,000.00"
"California, Oregon","$93,000.00","$148,250.00","$200,000.00"


### Bonus #2

Filter out the non-single-states (e.g., `'California, Colorado'`) in the most elegant way possible (i.e., *not* by blacklisting all the bad values).

In [ ]:
# Interestingly, at least one of the index values appears to be of type int...requiring the str type check
single_state_filter = summary_df.index.map(lambda x: len(x.split(", ")) == 1 if type(x) == str else False)
# Includes the District of Columbia so 51 different values
summary_df = summary_df[single_state_filter]
summary_df.style.format(df_formatting)

,min,mean,max
state,,,
California,$0.00,"$114,624.88","$875,000.00"
Washington,$72.00,"$107,290.97","$1,260,000.00"
District of Columbia,$40.00,"$106,750.16","$1,334,782.00"
New York,$80.00,"$105,338.96","$3,000,000.00"
Alabama,$54.00,"$102,140.20","$3,600,000.00"
New Jersey,"$14,850.00","$101,300.94","$5,000,044.00"
Massachusetts,$155.00,"$98,808.27","$1,650,000.00"
Virginia,$57.00,"$94,790.33","$1,300,000.00"
Connecticut,$0.00,"$93,766.44","$1,900,000.00"


In [ ]:
# Df with reset index although using state indices may be more useful
summary_df = summary_df.reset_index()
summary_df.style.format(df_formatting)

,state,min,mean,max
0,California,$0.00,"$114,624.88","$875,000.00"
1,Washington,$72.00,"$107,290.97","$1,260,000.00"
2,District of Columbia,$40.00,"$106,750.16","$1,334,782.00"
3,New York,$80.00,"$105,338.96","$3,000,000.00"
4,Alabama,$54.00,"$102,140.20","$3,600,000.00"
5,New Jersey,"$14,850.00","$101,300.94","$5,000,044.00"
6,Massachusetts,$155.00,"$98,808.27","$1,650,000.00"
7,Virginia,$57.00,"$94,790.33","$1,300,000.00"
8,Connecticut,$0.00,"$93,766.44","$1,900,000.00"
9,Illinois,$0.00,"$90,387.41","$1,100,000.00"


### Bonus #3

Show the quantiles instead of just minimum, mean, and maximum―say 0%, 5%, 25%, 50%, 75%, 95%, and 100%. Outliers may be deceiving.

Sort by whatever interests you―like say the *50th* percentile.

And throw in a count by state too. It would be interesting to know how many data points contribute to the figures for each state. (*Hint: your nice formatting from Bonus #1 might not work this time around.* 😜)

In [ ]:
# returns a function that takes in a Series and get the specified percentile of it
# adapted from: Andy Hayden's solution @ https://stackoverflow.com/questions/17578115/pass-percentiles-to-pandas-agg-function
def percentile(n):
    def percentile_factory(x):
        return x.quantile(n)
    # change name of manufactured function to be related to the percentile requested so that column names created by agg are descriptive rather than percentile_factory
    percentile_factory.__name__ = f"percentile_{n*100:02.0f}"
    return percentile_factory

percentile_summary_df = salary_us_2021_2022_df.groupby("state")["annual_salary"].agg([percentile(0), percentile(0.05), percentile(0.25), percentile(0.5), percentile(0.75), percentile(0.95), percentile(1), "count"]).sort_values(by="percentile_50", ascending=False)
# remove multi state results
single_state_filter2 = percentile_summary_df.index.map(lambda x: len(x.split(", ")) == 1 if type(x) == str else False)
percentile_summary_df = percentile_summary_df[single_state_filter2].reset_index()

In [ ]:
percentile_summary_df.style.format({col_name: money_format for col_name in percentile_summary_df.columns if col_name not in ["count", "state"]})

,state,percentile_00,percentile_05,percentile_25,percentile_50,percentile_75,percentile_95,percentile_100,count
0,California,$0.00,"$42,000.00","$72,000.00","$100,440.00","$147,000.00","$220,000.00","$875,000.00",2579
1,Washington,$72.00,"$40,000.00","$65,000.00","$91,000.00","$135,000.00","$200,600.00","$1,260,000.00",1177
2,District of Columbia,$40.00,"$48,000.00","$66,800.00","$90,000.00","$125,000.00","$189,650.00","$1,334,782.00",968
3,New York,$80.00,"$41,130.00","$64,503.75","$89,800.00","$127,922.00","$210,350.00","$3,000,000.00",2154
4,Massachusetts,$155.00,"$42,000.00","$64,000.00","$86,000.00","$120,000.00","$180,900.00","$1,650,000.00",1512
5,Maryland,$0.00,"$40,000.00","$60,000.00","$82,500.00","$110,000.00","$165,000.00","$353,200.00",558
6,Connecticut,$0.00,"$33,745.00","$62,000.00","$82,000.00","$100,000.00","$163,000.00","$1,900,000.00",235
7,Delaware,"$35,000.00","$41,704.00","$58,869.75","$81,322.50","$105,750.00","$164,247.50","$220,000.00",46
8,Virginia,$57.00,"$37,320.00","$58,750.00","$81,000.00","$115,000.00","$180,000.00","$1,300,000.00",777
9,Oregon,$0.00,"$36,340.00","$59,625.00","$78,500.00","$110,000.00","$185,000.00","$320,000.00",618
